In [2]:
%pip install pyswarms
%pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 KB 6.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from pyswarms.single.global_best import GlobalBestPSO
import numpy as np

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize and reshape the data
x_train = x_train / 255.0
x_test = x_test / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Define the optimization function
def cnn_loss(p):
    # Set the hyperparameters
    filters = int(p[0])
    kernel_size = (int(p[1]), int(p[2]))
    pool_size = (int(p[3]), int(p[4]))
    units = int(p[5])

    # Define the model
    model = Sequential()
    model.add(Conv2D(filters=filters, kernel_size=kernel_size, input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Flatten())
    model.add(Dense(units=units, activation='relu'))
    model.add(tf.keras.layers.Dense(units=10, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Fit and evaluate the model
    model.fit(x_train, y_train, epochs=5, batch_size=32)
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

    # Return the negative loss as the optimization function minimizes the cost
    return -loss

def optimization_function(p):
  lossVector = np.apply_along_axis(cnn_loss, axis=1, arr=p)

  return lossVector

# Set the search space
search_space = [
    (16, 64),  # filters
    (3, 5),    # kernel_size_x
    (3, 5),    # kernel_size_y
    (2, 4),    # pool_size_x
    (2, 4),    # pool_size_y
    (32, 256), # units
]

# Set the optimization options
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

# Set the bounds
lb = [a for a, _ in search_space]
ub = [b for _, b in search_space]
bounds = (lb, ub)
dimensions = len(lb)

# Initialize the optimizer
optimizer = GlobalBestPSO(n_particles=10, dimensions=dimensions, options=options, bounds=bounds)

# Run the optimization
cost, pos = optimizer.optimize(optimization_function, iters=10)

# Print the results
print("Best loss: ", -cost)
print("Best hyperparameters: ", pos)
print("")
print("Best conv layer filter number: " + str(int(pos[0])) )
print("Best conv layer kernel size: " + str((int(pos[1]), int(pos[2]))) )
print("Best pool layer size: " + str((int(pos[3]), int(pos[4]))) )
print("Best fully connected layer (dense) units: " + str(int(pos[5])) )



2023-01-13 09:01:17,731 - pyswarms.single.global_best - INFO - Optimize for 10 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/10

1875/1875 [==============================] - 37s 20ms/step - loss: 0.1863 - accuracy: 0.9454


pyswarms.single.global_best:  10%|█         |1/10, best_cost=-.0983

1875/1875 [==============================] - 31s 16ms/step - loss: 0.1731 - accuracy: 0.9491


pyswarms.single.global_best:  20%|██        |2/10, best_cost=-.0983

...

1875/1875 [==============================] - 41s 22ms/step - loss: 0.1997 - accuracy: 0.9420


pyswarms.single.global_best:  90%|█████████ |9/10, best_cost=-.116

1875/1875 [==============================] - 29s 15ms/step - loss: 0.2155 - accuracy: 0.9373


pyswarms.single.global_best: 100%|██████████|10/10, best_cost=-.116
2023-01-13 09:30:08,318 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.11584754288196564, best pos: [18.63463968  4.12898595  4.77082691  2.43070876  2.79851433 43.05663372]


Best loss:  0.11584754288196564
Best hyperparameters:  [18.63463968  4.12898595  4.77082691  2.43070876  2.79851433 43.05663372]

Best conv layer filter number: 18
Best conv layer kernel size: (4, 4)
Best pool layer size: (2, 2)
Best fully connected layer (dense) units: 43
